In [63]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 6.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.1 MB 9.3 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 11.2 MB/s eta 0:00:01
   ----------------------------------- ---- 10.0/11.1 MB 13.8 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 13.9 MB/s eta 0:00:00


In [64]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder


In [3]:
current_dir = os.getcwd()

# Original Dataset 
filename = 'df_processed.csv'
filepath = os.path.normpath(os.path.join(current_dir, '../data/processed/', filename))


chunk_size = 10000
chunks = []

for chunk in pd.read_csv(filepath, chunksize=chunk_size):
    chunk.drop_duplicates(inplace=True)
    chunk.dropna(inplace=True)
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

In [18]:
df.dtypes

p_recall             float64
timestamp              int64
delta                  int64
user_id               object
learning_language     object
ui_language           object
lexeme_id             object
history_seen           int64
history_correct        int64
session_seen           int64
session_correct        int64
h_recall             float64
dtype: object

In [9]:
df_users = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'users_behaviur.csv')))
df_words = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'word_complexity_features.csv')), sep='\t')

In [20]:
df.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,session_correct,h_recall
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,1,1.000000
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,6,0.750000
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,3,0.833333
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,8,0.833333
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,4,0.750000


In [29]:
df_users.drop(columns=['learning_language', 'ui_language'], inplace=True)

In [48]:
df['lang_combination'] = df['ui_language'] + '-' + df['learning_language']

In [49]:
df_1 = df.merge(df_words, on = 'lexeme_id', how='inner')

In [50]:
df_1

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,...,lang_combination,gender,tense,POS,def,person,number,word,word_len,SUBTLEX
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,...,en-es,NaN,NaN,pr,NaN,NaN,NaN,bajo,4,111241.0
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,...,en-de,NaN,pri,vblex,NaN,p3,sg,lernt,5,3391.0
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,...,en-de,f,NaN,det,def,NaN,sg,die,3,2484854.0
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,...,en-de,m,NaN,n,NaN,NaN,sg,mann,4,222707.0
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,...,en-de,f,NaN,n,NaN,NaN,sg,frau,4,143725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,6,4,5,...,it-en,NaN,NaN,det,def,NaN,sp,the,3,22761659.0
12509997,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,4,4,5,...,it-en,NaN,pres,vblex,NaN,NaN,NaN,eat,3,197654.0
12509998,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,4,4,4,...,it-en,NaN,NaN,n,NaN,NaN,sg,bread,5,24828.0
12509999,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,3,2,5,...,it-en,NaN,pres,vblex,NaN,NaN,NaN,drink,5,182580.0


In [51]:
df_2 = df_1.merge(df_users, on = ['user_id', 'lang_combination'], how='left')

In [53]:
df_2.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,...,def,person,number,word,word_len,SUBTLEX,avg_delta,avg_user_p_recall,std_delta,avg_h_recall
0,1.000000,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,3,3,1,...,NaN,NaN,NaN,bajo,4,111241.0,2.475405e+06,0.885194,2.879771e+06,0.954897
1,1.000000,1362082044,5963,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,8,6,6,...,NaN,p3,sg,lernt,5,3391.0,3.104417e+03,0.614120,2.977079e+03,0.890225
2,0.750000,1362082044,5963,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,6,5,4,...,def,NaN,sg,die,3,2484854.0,3.104417e+03,0.614120,2.977079e+03,0.890225
3,0.888889,1362082044,5963,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,6,5,9,...,NaN,NaN,sg,mann,4,222707.0,3.104417e+03,0.614120,2.977079e+03,0.890225
4,0.800000,1362082044,5963,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,8,6,5,...,NaN,NaN,sg,frau,4,143725.0,3.104417e+03,0.614120,2.977079e+03,0.890225


In [55]:
df_2.columns

Index(['p_recall', 'timestamp', 'delta', 'user_id', 'learning_language',
       'ui_language', 'lexeme_id', 'history_seen', 'history_correct',
       'session_seen', 'session_correct', 'h_recall', 'lang_combination',
       'gender', 'tense', 'POS', 'def', 'person', 'number', 'word', 'word_len',
       'SUBTLEX', 'avg_delta', 'avg_user_p_recall', 'std_delta',
       'avg_h_recall'],
      dtype='object')

In [57]:
### PREPARE DATA FOR ML 
dff = df_2.drop(columns=['timestamp', 'lexeme_id', 'word', 'user_id'])

In [59]:
dff.columns

Index(['p_recall', 'delta', 'learning_language', 'ui_language', 'history_seen',
       'history_correct', 'session_seen', 'session_correct', 'h_recall',
       'lang_combination', 'gender', 'tense', 'POS', 'def', 'person', 'number',
       'word_len', 'SUBTLEX', 'avg_delta', 'avg_user_p_recall', 'std_delta',
       'avg_h_recall'],
      dtype='object')

In [67]:
categorical_cols = dff.select_dtypes(include=['object']).columns.tolist()

# Replace NaN values with a placeholder for categorical columns

dff[categorical_cols] = dff[categorical_cols].fillna('Null')

# One-hot encode categorical variables
ohe = OneHotEncoder(sparse_output=False)
ohe_data = ohe.fit_transform(dff[categorical_cols])
ohe_df = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(categorical_cols))
df_encoded = pd.concat([dff.select_dtypes(exclude='O'), ohe_df], axis=1)

# Drop the original categorical columns
# df_encoded = df_encoded.drop(categorical_columns, axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
numerical_cols = ['word_len', 'SUBTLEX', 'avg_delta']
scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

X = df_encoded.drop('p_recall', axis=1)  # Features
y = df_encoded['p_recall'] 